# define necessary functions

In [4]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
import numpy as np
from mtcnn.mtcnn import MTCNN
import cv2
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.externals import joblib

def face_extract(filename,required_size=(160,160)):  
    image = Image.open(filename)    
    image = image.convert('RGB')                # convert to RGB, if needed    
    image = np.asarray(image)   
    detector = MTCNN()                          # create the detector, using default weights  
    results = detector.detect_faces(image)     # detect faces in the image    
    if len(results):
        faces,boxes=[],[]
        for i in range(len(results)):
            x1, y1, width, height = results[i]['box']    # extract the bounding box from the first face    
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            face = image[y1:y2, x1:x2]                  # extract the face
            face=cv2.resize(face,required_size)
            faces.append(face)
            boxes.append([x1,y1,width,height])
        return faces,boxes
    else:
        return ([],[]) 
    
# load images and extract faces for all images in a directory
def load_faces(directory):
    faces_lst = list()  
    for filename in listdir(directory):      
        path = directory + filename
        faces,_ = face_extract(path)                # get face
        if len(faces):
            faces_lst.append(faces[0])
    return faces_lst


# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    people_dict={}
    for i,subdir in enumerate(listdir(directory)):
        path = directory + subdir + '/'        
        if not isdir(path):               # skip any files that might be in the dir
            continue
        faces = load_faces(path)          # load all faces in the subdirectory
        people_dict.update({i:subdir})    # create key to people dictionary
        print('loaded %d images for person: %s' % (len(faces), subdir))
        labels=[subdir for _ in range(len(faces))]
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y),people_dict

def face_to_embedding(face,model):
        face = face.astype('float32')      
        mean, std = face.mean(), face.std()
        face_norm = (face - mean) / std        # normalize face pixels
        face_norm = np.expand_dims(face_norm, axis=0)       
        return model.predict(face_norm)        # make prediction to get embedding and return

# create new database and train the system

In [5]:

# load train dataset
x_train,y_train,people_dict = load_dataset('my_dataset/train/')
print('x_train shape:',x_train.shape,'\ny_train shape:', y_train.shape)

people_to_key_dict={v:k for k,v in people_dict.items()}

#load pre-trained facenet model
embed_model=load_model('facenet_model/facenet_keras.h5')
print('model is loaded successfully')

#convert face to embedding
x_train_embed=[]
y_train_ls=[]
for i in range(x_train.shape[0]):
    embed=face_to_embedding(x_train[i],embed_model)
    x_train_embed.append(embed)
    y_train_ls.append(people_to_key_dict[y_train[i]])

x_train_embed=np.asarray(x_train_embed).squeeze()
y_train=np.asarray(y_train_ls).squeeze()

# save embeddings in compressed format
np.savez_compressed('database/people-embeddings.npz',x_train_embed, y_train)
#save people dictionary 
joblib.dump(people_dict,'database/people_dictionary.pkl')
print('embeddings & people dictionary are saved successfully')

#normalize the embeddings
l2_normalizer=Normalizer(norm='l2')
x_train_embed=l2_normalizer.transform(x_train_embed)

#shuffle the data
np.random.seed(0)
np.random.shuffle(x_train_embed)
np.random.seed(0)
np.random.shuffle(y_train)

#create SVC model and fit 
FR_model = SVC(kernel='linear', probability=True)
FR_model.fit(x_train_embed,y_train)

# predict
yhat_train = FR_model.predict(x_train_embed)

score_train = accuracy_score(y_train, yhat_train)
print('Accuracy: train=%.3f' % (score_train*100))

#save svc model
joblib.dump(FR_model,'model/FR_model.pkl')
print('System is trained for face recognition sucessfully')

loaded 26 images for person: Aamir
loaded 22 images for person: Akshay
loaded 20 images for person: Faiz
loaded 13 images for person: Imran
loaded 17 images for person: salman
loaded 25 images for person: salman khan
loaded 13 images for person: shahrukh
x_train shape: (136, 160, 160, 3) 
y_train shape: (136,)
model is loaded successfully
embeddings & people dictionary are saved successfully
Accuracy: train=100.000
System is trained for face recognition sucessfully


## add new person in the existing database and train the system

In [12]:
##load the model and people dictionary
#FR_model=joblib.load('model/FR_model.pkl')
#embed_model=load_model('facenet_model/facenet_keras.h5')
people_dict=joblib.load('database/people_dictionary.pkl')

In [17]:
#let's add another actor
person_name='Hritik'
directory_path='new_person/Hritik/'

temp_dict=people_dict.copy()

def add_new_person(person_name,dir_path):
    new_key=len(temp_dict)
    temp_dict[new_key]=person_name

    #load faces
    faces=load_faces(dir_path)

    new_embed_set=[]
    new_label_set=[]
    for i in range(len(faces)):
        embed=face_to_embedding(faces[i],embed_model)
        new_embed_set.append(embed)
        new_label_set.append(new_key)

    new_embed_set=np.array(new_embed_set).squeeze()
    new_label_set=np.array(new_label_set).squeeze()


    data=np.load('database/people-embeddings.npz')
    x_train_embed,y_train=data['arr_0'],data['arr_1']
    x_train_embed=np.vstack((x_train_embed,new_embed_set))
    y_train=np.concatenate((y_train,new_label_set))
    
    #normalize the embeddings
    l2_normalizer=Normalizer(norm='l2')
    x_train_embed=l2_normalizer.transform(x_train_embed)

    #shuffle the data
    np.random.seed(0)
    np.random.shuffle(x_train_embed)
    np.random.seed(0)
    np.random.shuffle(y_train)

    #create SVC model and fit 
    FR_model = SVC(kernel='linear', probability=True)
    FR_model.fit(x_train_embed,y_train)

    # predict
    yhat_train = FR_model.predict(x_train_embed)

    score_train = accuracy_score(y_train, yhat_train)
    print('Accuracy: train=%.3f' % (score_train*100))
    
    people_dict=temp_dict.copy()
    #save people dictionary 
    joblib.dump(people_dict,'database/people_dictionary.pkl')
    
    # save embeddings in compressed format
    np.savez_compressed('database/people-embeddings.npz',x_train_embed, y_train)
    
    #save svc model
    joblib.dump(FR_model,'model/FR_model.pkl')
    
    print('%s is added in the database sucessfully'%(person_name))
    
add_new_person(person_name,directory_path)
people_dict=temp_dict

Accuracy: train=100.000
Hritik is added in the database sucessfully


# Test pre-trained model

In [2]:
#load the model and people dictionary
FR_model=joblib.load('model/FR_model.pkl')
embed_model=load_model('facenet_model/facenet_keras.h5')
people_dict=joblib.load('database/people_dictionary.pkl')

C:\Users\Admin\Anaconda3\envs\tf\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


### on image

In [21]:
image_path='faiz.jpg'
def find_people(img_path,model,embed_model):
    faces,boxes=face_extract(img_path)
    if len(faces):
        img=cv2.imread(image_path,1)
        for i in range(len(faces)):
            face,(x,y,w,h)=faces[i],boxes[i]
            embed=face_to_embedding(face,embed_model)
            Nembed=Normalizer(norm='l2').transform(embed)
            pred=model.predict_proba(Nembed)
            prob=pred[0][np.argmax(pred[0])]
            label=people_dict[np.argmax(pred[0])]+(" %.2f"%(prob*100)) if prob >.65 else 'unknown'                      
            cv2.rectangle(img,(x,y), (x+w,y+h), (255,255,0), 1)
            cv2.putText(img,label, (int(x), int(y-15)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        try:

            cv2.imshow('image',img)
            if cv2.waitKey(0)==ord('q'):
                cv2.destroyAllWindows() 
        finally:
            cv2.destroyAllWindows()
    else:
        print('there is no face in the image')

find_people(image_path,FR_model,embed_model)


### at real-time

In [22]:
face_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml') #extract face feature
cap=cv2.VideoCapture(0)
try: 
    while True:
        _,frame=cap.read()
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        if len(faces):
            for face in faces:
                    (x,y,w,h)=face
                    face_img=frame[y:y+h,x:x+w]
                    face_img=cv2.resize(face_img,(160,160))
                    embed=face_to_embedding(face_img,embed_model)         
                    Nembed=Normalizer(norm='l2').transform(embed.reshape(1,-1))
                    pred=FR_model.predict_proba(Nembed)
                    clas=np.argmax(pred[0])
                    prob=pred[0][clas]
                    if  prob >.45:
                        label=people_dict[clas]+(" %.2f"%(prob*100))
                    else:
                        label='unknown'
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,0), 1)
                    cv2.putText(frame,label, (int(x), int(y-15)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
        cv2.imshow('Face Recognition',frame)
        if cv2.waitKey(1)==ord('q'):
            break   
finally:            
    cap.release()
    cv2.destroyAllWindows()
